In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as tf
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

In [2]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2),
            nn.Flatten(),
        )

        self.fc = nn.Sequential(
            nn.Linear(288, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.convolution(x)
        x = self.fc(x)
        x = tf.dropout(x, p=0.2, training=self.training)
        return x

In [3]:
mnist_batch_size=50
mnist_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data/mnist_train', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=mnist_batch_size,
    shuffle=True
)

mnist_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist_test', train=False, download=True, transform=transforms.ToTensor()),
    batch_size=100
)

In [4]:
def train(model, train_loader, batch_size, epoch_cnt, optimizer):
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    model.train()
    train_loss = []
    train_acc = []
    batch_size = 50
    i = 0
    for epoch in range(epoch_cnt):
        print('epoch: {}'.format(epoch))
        for data, target in train_loader:
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = tf.cross_entropy(output, target)
            loss.backward()    # calc gradients
            train_loss.append(loss.data.item())
            optimizer.step()   # update gradients
            prediction = output.data.max(1)[1]   # first column has actual prob.
            accuracy = prediction.eq(target.data).sum()/batch_size*100
            train_acc.append(accuracy)
            if i % 300 == 0:
                print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data.item(), accuracy))
            i += 1
    return [train_acc, train_loss]

In [5]:
mnist_model = MnistModel()
optimizer = optim.Adam(mnist_model.parameters(), lr=0.0001)
[mnist_acc, mnist_loss] = train(model=mnist_model, train_loader=mnist_train_loader, batch_size=mnist_batch_size, epoch_cnt=10, optimizer=optimizer)

epoch: 0
Train Step: 0	Loss: 2.307	Accuracy: 12.000
Train Step: 300	Loss: 0.654	Accuracy: 80.000
Train Step: 600	Loss: 0.492	Accuracy: 86.000
Train Step: 900	Loss: 0.230	Accuracy: 96.000
epoch: 1
Train Step: 1200	Loss: 0.407	Accuracy: 84.000
Train Step: 1500	Loss: 0.263	Accuracy: 88.000
Train Step: 1800	Loss: 0.390	Accuracy: 76.000
Train Step: 2100	Loss: 0.549	Accuracy: 82.000
epoch: 2
Train Step: 2400	Loss: 0.332	Accuracy: 80.000
Train Step: 2700	Loss: 0.377	Accuracy: 82.000
Train Step: 3000	Loss: 0.253	Accuracy: 92.000
Train Step: 3300	Loss: 0.450	Accuracy: 82.000
epoch: 3
Train Step: 3600	Loss: 0.180	Accuracy: 94.000
Train Step: 3900	Loss: 0.267	Accuracy: 88.000
Train Step: 4200	Loss: 0.190	Accuracy: 92.000
Train Step: 4500	Loss: 0.235	Accuracy: 84.000
epoch: 4
Train Step: 4800	Loss: 0.290	Accuracy: 88.000
Train Step: 5100	Loss: 0.315	Accuracy: 86.000
Train Step: 5400	Loss: 0.405	Accuracy: 88.000
Train Step: 5700	Loss: 0.243	Accuracy: 86.000
epoch: 5
Train Step: 6000	Loss: 0.301	Acc

In [6]:
def eval(model, test_loader):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        prediction = output.data.max(1)[1]
        correct += prediction.eq(target.data).sum()
    print('\nTest set: Accuracy: {:.2f}%'.format(100. * correct / len(test_loader.dataset)))

In [7]:
eval(model=mnist_model, test_loader=mnist_test_loader)

/var/folders/gg/yt92tvfx4dg7s1cn2l6wtk09xzv94b/T/ipykernel_13741/2084227670.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Accuracy: 98.85%


In [8]:
writer = SummaryWriter()

for step in range(len(mnist_loss)):
    writer.add_scalar('train/mnist_loss', mnist_loss[step], step)
    writer.add_scalar('train/mnist_accuracy', mnist_acc[step], step)

In [9]:
fashion_mnist_batch_size=50
fashion_mnist_train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(root='./data/fashion_mnist_train', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=fashion_mnist_batch_size,
    shuffle=True
)

fashion_mnist_test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data/fashion_mnist_test', train=False, download=True, transform=transforms.ToTensor()),
    batch_size=100
)

In [10]:
fashion_mnist_model = MnistModel()
fashion_mnist_optimizer = optim.Adam(fashion_mnist_model.parameters(), lr=0.0001)
[fashion_mnist_acc, fashion_mnist_loss] = train(
    model=fashion_mnist_model,
    train_loader=fashion_mnist_train_loader,
    batch_size=fashion_mnist_batch_size,
    epoch_cnt=10,
    optimizer=fashion_mnist_optimizer
)

epoch: 0
Train Step: 0	Loss: 2.318	Accuracy: 10.000
Train Step: 300	Loss: 0.993	Accuracy: 60.000
Train Step: 600	Loss: 1.087	Accuracy: 54.000
Train Step: 900	Loss: 0.741	Accuracy: 64.000
epoch: 1
Train Step: 1200	Loss: 0.844	Accuracy: 64.000
Train Step: 1500	Loss: 0.977	Accuracy: 64.000
Train Step: 1800	Loss: 0.699	Accuracy: 74.000
Train Step: 2100	Loss: 0.818	Accuracy: 68.000
epoch: 2
Train Step: 2400	Loss: 0.793	Accuracy: 62.000
Train Step: 2700	Loss: 0.832	Accuracy: 68.000
Train Step: 3000	Loss: 0.705	Accuracy: 72.000
Train Step: 3300	Loss: 0.469	Accuracy: 82.000
epoch: 3
Train Step: 3600	Loss: 0.632	Accuracy: 78.000
Train Step: 3900	Loss: 0.920	Accuracy: 76.000
Train Step: 4200	Loss: 0.667	Accuracy: 72.000
Train Step: 4500	Loss: 0.542	Accuracy: 84.000
epoch: 4
Train Step: 4800	Loss: 0.487	Accuracy: 84.000
Train Step: 5100	Loss: 0.871	Accuracy: 70.000
Train Step: 5400	Loss: 0.711	Accuracy: 82.000
Train Step: 5700	Loss: 0.641	Accuracy: 68.000
epoch: 5
Train Step: 6000	Loss: 0.558	Acc

In [11]:
eval(model=fashion_mnist_model, test_loader=fashion_mnist_test_loader)

/var/folders/gg/yt92tvfx4dg7s1cn2l6wtk09xzv94b/T/ipykernel_13741/2084227670.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Accuracy: 88.22%


In [12]:
writer = SummaryWriter()

for step in range(len(fashion_mnist_loss)):
    writer.add_scalar('train/fashion_mnist_loss', fashion_mnist_loss[step], step)
    writer.add_scalar('train/fashion_mnist_accuracy', fashion_mnist_acc[step], step)

In [13]:
torch.save(mnist_model, 'mnist_model.pt')
torch.save(fashion_mnist_model, 'fashion_mnist_model.pt')

In [14]:
fashion_mnist_pt_model = torch.load('mnist_model.pt')
fashion_mnist_pt_optimizer = optim.Adam(fashion_mnist_pt_model.parameters(), lr=0.0001)
[fashion_mnist_pt_acc, fashion_mnist_pt_loss] = train(
    model=fashion_mnist_pt_model,
    train_loader=fashion_mnist_train_loader,
    batch_size=fashion_mnist_batch_size,
    epoch_cnt=10,
    optimizer=fashion_mnist_pt_optimizer
)

epoch: 0
Train Step: 0	Loss: 4.879	Accuracy: 8.000
Train Step: 300	Loss: 0.885	Accuracy: 66.000
Train Step: 600	Loss: 0.805	Accuracy: 74.000
Train Step: 900	Loss: 0.790	Accuracy: 68.000
epoch: 1
Train Step: 1200	Loss: 0.902	Accuracy: 64.000
Train Step: 1500	Loss: 0.810	Accuracy: 72.000
Train Step: 1800	Loss: 0.814	Accuracy: 70.000
Train Step: 2100	Loss: 0.547	Accuracy: 78.000
epoch: 2
Train Step: 2400	Loss: 0.863	Accuracy: 72.000
Train Step: 2700	Loss: 0.417	Accuracy: 82.000
Train Step: 3000	Loss: 0.726	Accuracy: 72.000
Train Step: 3300	Loss: 0.458	Accuracy: 74.000
epoch: 3
Train Step: 3600	Loss: 0.592	Accuracy: 80.000
Train Step: 3900	Loss: 0.658	Accuracy: 72.000
Train Step: 4200	Loss: 0.516	Accuracy: 76.000
Train Step: 4500	Loss: 0.893	Accuracy: 66.000
epoch: 4
Train Step: 4800	Loss: 0.500	Accuracy: 78.000
Train Step: 5100	Loss: 0.665	Accuracy: 72.000
Train Step: 5400	Loss: 0.613	Accuracy: 84.000
Train Step: 5700	Loss: 0.586	Accuracy: 68.000
epoch: 5
Train Step: 6000	Loss: 0.578	Accu

In [15]:
eval(model=fashion_mnist_pt_model, test_loader=fashion_mnist_test_loader)

/var/folders/gg/yt92tvfx4dg7s1cn2l6wtk09xzv94b/T/ipykernel_13741/2084227670.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Accuracy: 88.89%


In [16]:
writer = SummaryWriter()

for step in range(len(fashion_mnist_pt_loss)):
    writer.add_scalar('train/fashion_mnist_pt_loss', fashion_mnist_pt_loss[step], step)
    writer.add_scalar('train/fashion_mnist_pt_accuracy', fashion_mnist_pt_acc[step], step)

In [17]:
fashion_mnist_freeze_model = torch.load('mnist_model.pt')
for param in fashion_mnist_freeze_model.convolution.parameters():
    param.requires_grad = False

fashion_mnist_freeze_optimizer = optim.Adam(fashion_mnist_freeze_model.parameters(), lr=0.0001)
[fashion_mnist_freeze_acc, fashion_mnist_freeze_loss] = train(
    model=fashion_mnist_freeze_model,
    train_loader=fashion_mnist_train_loader,
    batch_size=fashion_mnist_batch_size,
    epoch_cnt=10,
    optimizer=fashion_mnist_freeze_optimizer
)

epoch: 0
Train Step: 0	Loss: 5.147	Accuracy: 14.000
Train Step: 300	Loss: 1.184	Accuracy: 64.000
Train Step: 600	Loss: 1.067	Accuracy: 58.000
Train Step: 900	Loss: 0.920	Accuracy: 56.000
epoch: 1
Train Step: 1200	Loss: 0.807	Accuracy: 72.000
Train Step: 1500	Loss: 1.084	Accuracy: 68.000
Train Step: 1800	Loss: 0.572	Accuracy: 78.000
Train Step: 2100	Loss: 0.787	Accuracy: 68.000
epoch: 2
Train Step: 2400	Loss: 0.740	Accuracy: 74.000
Train Step: 2700	Loss: 0.973	Accuracy: 58.000
Train Step: 3000	Loss: 0.885	Accuracy: 64.000
Train Step: 3300	Loss: 0.933	Accuracy: 56.000
epoch: 3
Train Step: 3600	Loss: 0.797	Accuracy: 64.000
Train Step: 3900	Loss: 0.723	Accuracy: 70.000
Train Step: 4200	Loss: 0.779	Accuracy: 66.000
Train Step: 4500	Loss: 0.791	Accuracy: 70.000
epoch: 4
Train Step: 4800	Loss: 0.700	Accuracy: 68.000
Train Step: 5100	Loss: 0.530	Accuracy: 78.000
Train Step: 5400	Loss: 0.663	Accuracy: 76.000
Train Step: 5700	Loss: 0.754	Accuracy: 68.000
epoch: 5
Train Step: 6000	Loss: 0.686	Acc

In [18]:
eval(model=fashion_mnist_freeze_model, test_loader=fashion_mnist_test_loader)

/var/folders/gg/yt92tvfx4dg7s1cn2l6wtk09xzv94b/T/ipykernel_13741/2084227670.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Accuracy: 84.29%


In [19]:
writer = SummaryWriter()

for step in range(len(fashion_mnist_freeze_loss)):
    writer.add_scalar('train/fashion_mnist_freeze_loss', fashion_mnist_freeze_loss[step], step)
    writer.add_scalar('train/fashion_mnist_freeze_accuracy', fashion_mnist_freeze_acc[step], step)

In [20]:

fashion_mnist_freeze_unfreeze_model = torch.load('mnist_model.pt')
for param in fashion_mnist_freeze_unfreeze_model.convolution.parameters():
    param.requires_grad = False

fashion_mnist_freeze_unfreeze_optimizer = optim.Adam(fashion_mnist_freeze_unfreeze_model.parameters(), lr=0.0001)
train(
    model=fashion_mnist_freeze_unfreeze_model,
    train_loader=fashion_mnist_train_loader,
    batch_size=fashion_mnist_batch_size,
    epoch_cnt=3,
    optimizer=fashion_mnist_freeze_unfreeze_optimizer
)

for param in fashion_mnist_freeze_unfreeze_model.convolution.parameters():
    param.requires_grad = True

[fashion_mnist_freeze_unfreeze_acc, fashion_mnist_freeze_unfreeze_loss] = train(
    model=fashion_mnist_freeze_unfreeze_model,
    train_loader=fashion_mnist_train_loader,
    batch_size=fashion_mnist_batch_size,
    epoch_cnt=7,
    optimizer=fashion_mnist_freeze_unfreeze_optimizer
)

epoch: 0
Train Step: 0	Loss: 5.363	Accuracy: 20.000
Train Step: 300	Loss: 1.271	Accuracy: 64.000
Train Step: 600	Loss: 1.069	Accuracy: 72.000
Train Step: 900	Loss: 0.702	Accuracy: 78.000
epoch: 1
Train Step: 1200	Loss: 0.855	Accuracy: 70.000
Train Step: 1500	Loss: 1.059	Accuracy: 48.000
Train Step: 1800	Loss: 0.827	Accuracy: 74.000
Train Step: 2100	Loss: 0.701	Accuracy: 72.000
epoch: 2
Train Step: 2400	Loss: 0.858	Accuracy: 60.000
Train Step: 2700	Loss: 0.994	Accuracy: 54.000
Train Step: 3000	Loss: 0.839	Accuracy: 68.000
Train Step: 3300	Loss: 0.778	Accuracy: 70.000
epoch: 0
Train Step: 0	Loss: 0.891	Accuracy: 74.000
Train Step: 300	Loss: 0.775	Accuracy: 70.000
Train Step: 600	Loss: 0.757	Accuracy: 66.000
Train Step: 900	Loss: 0.548	Accuracy: 76.000
epoch: 1
Train Step: 1200	Loss: 0.651	Accuracy: 76.000
Train Step: 1500	Loss: 0.601	Accuracy: 78.000
Train Step: 1800	Loss: 0.570	Accuracy: 74.000
Train Step: 2100	Loss: 0.578	Accuracy: 74.000
epoch: 2
Train Step: 2400	Loss: 0.608	Accuracy:

In [21]:
eval(model=fashion_mnist_freeze_unfreeze_model, test_loader=fashion_mnist_test_loader)

/var/folders/gg/yt92tvfx4dg7s1cn2l6wtk09xzv94b/T/ipykernel_13741/2084227670.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Accuracy: 88.19%


In [22]:
writer = SummaryWriter()

for step in range(len(fashion_mnist_freeze_unfreeze_loss)):
    writer.add_scalar('train/fashion_mnist_freeze_unfreeze_loss', fashion_mnist_freeze_unfreeze_loss[step], step)
    writer.add_scalar('train/fashion_mnist_freeze_unfreeze_accuracy', fashion_mnist_freeze_unfreeze_acc[step], step)

In [2]:
%load_ext tensorboard
%tensorboard --logdir runs